In [46]:
import pandas as pd
import os
base_address = r"\\neptune.radsci.uci.edu\NeptuneData\Datasets\Rotterdam"
address = r"\training\dataset00\vessel0\reference.txt"


In [18]:
data = pd.read_csv(base_address+address, sep=" ",names = ["X", "Y","Z", "a","b"])
data


,X,Y,Z,a,b
0,68.6513,57.7580,75.2850,2.544800,0.259307
1,68.6363,57.7321,75.2869,2.544480,0.258062
2,68.6213,57.7062,75.2889,2.544160,0.257561
3,68.6064,57.6802,75.2908,2.543840,0.257103
4,68.5914,57.6543,75.2928,2.543520,0.256715
...,...,...,...,...,...
7349,155.5690,54.0919,34.2338,0.828176,0.624076
7350,155.5650,54.0630,34.2269,0.826917,0.652115
7351,155.5610,54.0342,34.2196,0.822742,0.680152
7352,155.5570,54.0054,34.2123,0.818567,0.708350


In [6]:
import SimpleITK as sitk
import numpy as np
def load_itk(filename):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(filename)

    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = sitk.GetArrayFromImage(itkimage)

    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    origin = np.array(list(reversed(itkimage.GetOrigin())))

    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itkimage.GetSpacing())))

    return ct_scan, origin, spacing

In [21]:
ct_scan, origin, spacing = load_itk(r"\\neptune.radsci.uci.edu\NeptuneData\Datasets\Rotterdam\training\dataset00\image00.mhd")
spacing

array([0.40000001, 0.36328101, 0.36328101])

In [53]:
def listdir_nohidden(path):
    myList = []
    for f in os.listdir(path):
        if not f.startswith('.'):
            myList.append(f)
    return myList      
def real_world_to_pixel(address, spacing):
    df = pd.read_csv(address + "\\" + "reference.txt",sep=" ",names = ["X", "Y","Z", "a","b"])
    for index, row in df.iterrows():
        row["X"] = int(round(row["X"]/spacing[0],0))
        row["Y"] = int(round(row["Y"]/spacing[1],0))
        row["Z"] = int(round(row["Z"]/spacing[1],0))
    df.to_csv(address + "\\" + "reference_voxel.csv")
def translate():
    base_address = r"\\neptune.radsci.uci.edu\NeptuneData\Datasets\Rotterdam"
    train_test_dire_list = ["training"]
    # training folder
    for data_directory in train_test_dire_list:
        first_address = base_address+ "\\"+ data_directory
        # dataSet folder
        for dataset in listdir_nohidden(first_address):
            second_address = first_address + "\\" + dataset
            # find mhd file and get spacing
            for file in listdir_nohidden(second_address):
                if file.endswith(".mhd"):
                    try:
                        ct_scan, origin, spacing = load_itk(second_address + "\\" + file)
                        for sub_dir in listdir_nohidden(second_address):
                            if (sub_dir in {"vessel0", "vessel1", "vessel2", "vessel3"} and 
                                "reference.txt" in listdir_nohidden(second_address + "\\" + sub_dir)):
                                real_world_to_pixel(second_address + "\\" + sub_dir, spacing)
                    except:
                        print("aa")

                                
                

In [54]:
translate()

aa
aa
aa
aa
aa
aa


171.3